### Bibliotecas e pacotes

In [2]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, round, sum, avg, max, min, when, countDistinct, count, date_format, current_date

In [3]:
#configurando sessão
from pyspark.sql import SparkSession
spark = SparkSession \
        .builder \
        .appName("Pos_Cash") \
        .master("local[*]") \
        .config("spark.driver.memory", "4g") \
        .config("spark.executor.memory", "4g") \
        .getOrCreate()

### Utilizando spark para ler os dados

In [4]:
dados_pos = spark.read.csv("../database/raw/POS_CASH_balance.csv", header=True, inferSchema=True)
dados_pos.createOrReplaceTempView("dados_pos")

In [5]:
dados_pos.show(5)

+----------+----------+--------------+--------------+---------------------+--------------------+------+----------+
|SK_ID_PREV|SK_ID_CURR|MONTHS_BALANCE|CNT_INSTALMENT|CNT_INSTALMENT_FUTURE|NAME_CONTRACT_STATUS|SK_DPD|SK_DPD_DEF|
+----------+----------+--------------+--------------+---------------------+--------------------+------+----------+
|   1803195|    182943|           -31|          48.0|                 45.0|              Active|     0|         0|
|   1715348|    367990|           -33|          36.0|                 35.0|              Active|     0|         0|
|   1784872|    397406|           -32|          12.0|                  9.0|              Active|     0|         0|
|   1903291|    269225|           -35|          48.0|                 42.0|              Active|     0|         0|
|   2341044|    334279|           -35|          36.0|                 35.0|              Active|     0|         0|
+----------+----------+--------------+--------------+---------------------+-----

### Verificando quantidade de linhas e colunas no dataframe

In [6]:
# Verificando a quantidade de linhas e colunas do DataFrame.

# Quantidade de linhas.
qtt_rows = dados_pos.count()

# Quantidade de colunas.
qtt_columns = len(dados_pos.columns)

# Quantidade de IDs únicos.
distinct_id_pos = spark.sql('''SELECT COUNT(DISTINCT `SK_ID_PREV`) as distinct_id_pos FROM dados_pos ''')
distinct_id_pos.createOrReplaceTempView("distinct_id_pos")

# Imprimir o resultado.
print(f'Quantidade de linhas do DataFrame: {qtt_rows}')
print(f'Quantidade de colunas do DataFrame: {qtt_columns}')
distinct_id_pos.show()

Quantidade de linhas do DataFrame: 10001358
Quantidade de colunas do DataFrame: 8
+---------------+
|distinct_id_pos|
+---------------+
|         936325|
+---------------+



### Função para renomear e padronizar palavras

In [7]:
#Substituir espaço por underline e colocar a palavra em letras maiúsculas.
import re

def tratamento_palavra(palavra):
    palavra_transformada = re.sub(r'\s', '_', palavra).upper()

    return palavra_transformada

### Renomeando e padronizando palavras da coluna `NAME_CONTRACT_STATUS`

In [8]:
# Extraindo os valores únicos diretamente do DataFrame
status = dados_pos.select('NAME_CONTRACT_STATUS').distinct().collect()

# Aplicando a função de tratamento de palavra a cada valor único
nova_lista = [tratamento_palavra(row['NAME_CONTRACT_STATUS']) for row in status]

# Exibindo a lista resultante
print(nova_lista)

['DEMAND', 'APPROVED', 'COMPLETED', 'RETURNED_TO_THE_STORE', 'ACTIVE', 'SIGNED', 'XNA', 'CANCELED', 'AMORTIZED_DEBT']


### Criando flags de janela temporal

In [9]:
df_temp_01 = spark.sql('''
SELECT
    *,
    CASE WHEN MONTHS_BALANCE >= -3 THEN 1 ELSE 0 END AS U3M,
    CASE WHEN MONTHS_BALANCE >= -6 THEN 1 ELSE 0 END AS U6M,
    CASE WHEN MONTHS_BALANCE >= -12 THEN 1 ELSE 0 END AS U12M,
    CASE WHEN MONTHS_BALANCE >= -24 THEN 1 ELSE 0 END AS U24M,
    CASE WHEN MONTHS_BALANCE >= -36 THEN 1 ELSE 0 END AS U36M
FROM
    dados_pos
ORDER BY
    SK_ID_PREV
''')

df_temp_01.createOrReplaceTempView('df_temp_01')
df_temp_01.show()

+----------+----------+--------------+--------------+---------------------+--------------------+------+----------+---+---+----+----+----+
|SK_ID_PREV|SK_ID_CURR|MONTHS_BALANCE|CNT_INSTALMENT|CNT_INSTALMENT_FUTURE|NAME_CONTRACT_STATUS|SK_DPD|SK_DPD_DEF|U3M|U6M|U12M|U24M|U36M|
+----------+----------+--------------+--------------+---------------------+--------------------+------+----------+---+---+----+----+----+
|   1000001|    158271|           -10|          12.0|                 12.0|              Active|     0|         0|  0|  0|   1|   1|   1|
|   1000001|    158271|            -8|           2.0|                  0.0|           Completed|     0|         0|  0|  0|   1|   1|   1|
|   1000001|    158271|            -9|          12.0|                 11.0|              Active|     0|         0|  0|  0|   1|   1|   1|
|   1000002|    101962|           -54|           4.0|                  4.0|              Active|     0|         0|  0|  0|   0|   0|   0|
|   1000002|    101962|           

### Criando variáveis de primeira camada

In [10]:
# Definindo as colunas para a agregação.
colunas_agregacao_total = df_temp_01.columns
colunas_agregacao_total.remove('SK_ID_CURR')
colunas_agregacao_total.remove('SK_ID_PREV')
colunas_agregacao_total.remove('MONTHS_BALANCE')
colunas_agregacao_total.remove('NAME_CONTRACT_STATUS')

# Defindo a lista de colunas de flags.
colunas_flags = ['U3M', 'U6M', 'U12M', 'U24M', 'U36M']

# Criando uma lista vazia.
expressoes_agregacao = []

# Iterando sobre as colunas e criando as variáveis explicativas com as agregações.
for coluna in colunas_agregacao_total:
  # Verifica se a coluna atual não é uma coluna de flag.
  if not any(flag in coluna for flag in colunas_flags):
    for flag in colunas_flags:
      if 'DPD' in coluna:
        expressoes_agregacao.append(round(sum(when(col(flag) == 1, col(coluna))), 2).alias(f'QT_TOT_{coluna.upper()}_{flag.upper()}_POS_CASH'))
        expressoes_agregacao.append(round(avg(when(col(flag) == 1, col(coluna))), 2).alias(f'QT_MED_{coluna.upper()}_{flag.upper()}_POS_CASH'))
        expressoes_agregacao.append(round(max(when(col(flag) == 1, col(coluna))), 2).alias(f'QT_MAX_{coluna.upper()}_{flag.upper()}_POS_CASH'))
        expressoes_agregacao.append(round(min(when(col(flag) == 1, col(coluna))), 2).alias(f'QT_MIN_{coluna.upper()}_{flag.upper()}_POS_CASH'))
      else:
        expressoes_agregacao.append(round(sum(when(col(flag) == 1, col(coluna))), 2).alias(f'VL_TOT_{coluna.upper()}_{flag.upper()}_POS_CASH'))
        expressoes_agregacao.append(round(avg(when(col(flag) == 1, col(coluna))), 2).alias(f'VL_MED_{coluna.upper()}_{flag.upper()}_POS_CASH'))
        expressoes_agregacao.append(round(max(when(col(flag) == 1, col(coluna))), 2).alias(f'VL_MAX_{coluna.upper()}_{flag.upper()}_POS_CASH'))
        expressoes_agregacao.append(round(min(when(col(flag) == 1, col(coluna))), 2).alias(f'VL_MIN_{coluna.upper()}_{flag.upper()}_POS_CASH'))

# Criando uma tupla com as variáveis criadas.
expressoes_agregacao = tuple(expressoes_agregacao)

# Aplicando as expressões de agregação.
df_temp_02 = df_temp_01.groupBy('SK_ID_PREV').agg(*expressoes_agregacao).orderBy('SK_ID_PREV')


# Quantidade e nome das variáveis criadas.
nomes_cols = df_temp_02.columns
nomes_cols_novas = nomes_cols[1:]
print('Quantidade Total de Variáveis Criadas:', len(df_temp_02.columns) - 1)
print('Nomes das Variáveis Criadas:', nomes_cols_novas)
print('')
print('')

# Quantidade de linhas do DataFrame.
num_rows_df = df_temp_02.count()

# Quantidade de colunas do DataFrame.
num_columns_df = len(df_temp_02.columns)

# Imprimir o resultado de número de linhas e colunas.
print(f'Quantidade de linhas do DataFrame: {num_rows_df}')
print(f'Quantidade de colunas do DataFrame: {num_columns_df}')
print('')
print('')

# Mostrando o novo DataFrame com as variáveis criadas.
df_temp_02.show(5, False)

Quantidade Total de Variáveis Criadas: 80
Nomes das Variáveis Criadas: ['VL_TOT_CNT_INSTALMENT_U3M_POS_CASH', 'VL_MED_CNT_INSTALMENT_U3M_POS_CASH', 'VL_MAX_CNT_INSTALMENT_U3M_POS_CASH', 'VL_MIN_CNT_INSTALMENT_U3M_POS_CASH', 'VL_TOT_CNT_INSTALMENT_U6M_POS_CASH', 'VL_MED_CNT_INSTALMENT_U6M_POS_CASH', 'VL_MAX_CNT_INSTALMENT_U6M_POS_CASH', 'VL_MIN_CNT_INSTALMENT_U6M_POS_CASH', 'VL_TOT_CNT_INSTALMENT_U12M_POS_CASH', 'VL_MED_CNT_INSTALMENT_U12M_POS_CASH', 'VL_MAX_CNT_INSTALMENT_U12M_POS_CASH', 'VL_MIN_CNT_INSTALMENT_U12M_POS_CASH', 'VL_TOT_CNT_INSTALMENT_U24M_POS_CASH', 'VL_MED_CNT_INSTALMENT_U24M_POS_CASH', 'VL_MAX_CNT_INSTALMENT_U24M_POS_CASH', 'VL_MIN_CNT_INSTALMENT_U24M_POS_CASH', 'VL_TOT_CNT_INSTALMENT_U36M_POS_CASH', 'VL_MED_CNT_INSTALMENT_U36M_POS_CASH', 'VL_MAX_CNT_INSTALMENT_U36M_POS_CASH', 'VL_MIN_CNT_INSTALMENT_U36M_POS_CASH', 'VL_TOT_CNT_INSTALMENT_FUTURE_U3M_POS_CASH', 'VL_MED_CNT_INSTALMENT_FUTURE_U3M_POS_CASH', 'VL_MAX_CNT_INSTALMENT_FUTURE_U3M_POS_CASH', 'VL_MIN_CNT_INSTALMEN

### Criando variáveis com base na coluna ``NAME_CONTRACT_STATUS``

In [11]:
# Definindo as colunas para agregação.
colunas_agregacao_total = df_temp_01.columns
colunas_agregacao_total.remove('SK_ID_CURR')
colunas_agregacao_total.remove('SK_ID_PREV')
colunas_agregacao_total.remove('MONTHS_BALANCE')
colunas_agregacao_total.remove('NAME_CONTRACT_STATUS')

# Defindo a lista de colunas de flags.
colunas_flags = ['U3M', 'U6M', 'U12M', 'U24M', 'U36M']

# Criando uma lista vazia.
expressoes_agregacao = []

# Iterando sobre as colunas e criando as variáveis explicativas com as agregações.
for categoria in nova_lista:
  for coluna in colunas_agregacao_total:
    # Verifica se a coluna atual não é uma coluna de flag.
    if not any(flag in coluna for flag in colunas_flags):
      for flag in colunas_flags:
        if 'DPD' in coluna:
          expressoes_agregacao.append(round(sum(when(col(flag) == 1, col(coluna))), 2).alias(f'QT_TOT_{coluna.upper()}_{flag.upper()}_{categoria}_POS_CASH'))
          expressoes_agregacao.append(round(avg(when(col(flag) == 1, col(coluna))), 2).alias(f'QT_MED_{coluna.upper()}_{flag.upper()}_{categoria}_POS_CASH'))
          expressoes_agregacao.append(round(max(when(col(flag) == 1, col(coluna))), 2).alias(f'QT_MAX_{coluna.upper()}_{flag.upper()}_{categoria}_POS_CASH'))
          expressoes_agregacao.append(round(min(when(col(flag) == 1, col(coluna))), 2).alias(f'QT_MIN_{coluna.upper()}_{flag.upper()}_{categoria}_POS_CASH'))
        else:
          expressoes_agregacao.append(round(sum(when(col(flag) == 1, col(coluna))), 2).alias(f'VL_TOT_{coluna.upper()}_{flag.upper()}_{categoria}_POS_CASH'))
          expressoes_agregacao.append(round(avg(when(col(flag) == 1, col(coluna))), 2).alias(f'VL_MED_{coluna.upper()}_{flag.upper()}_{categoria}_POS_CASH'))
          expressoes_agregacao.append(round(max(when(col(flag) == 1, col(coluna))), 2).alias(f'VL_MAX_{coluna.upper()}_{flag.upper()}_{categoria}_POS_CASH'))
          expressoes_agregacao.append(round(min(when(col(flag) == 1, col(coluna))), 2).alias(f'VL_MIN_{coluna.upper()}_{flag.upper()}_{categoria}_POS_CASH'))

# Criando uma tupla com as variáveis criadas.
expressoes_agregacao = tuple(expressoes_agregacao)

# Aplicando as expressões de agregação.
df_temp_03 = df_temp_01.groupBy('SK_ID_PREV').agg(*expressoes_agregacao).orderBy('SK_ID_PREV')


# Quantidade e nome das variáveis criadas.
nomes_cols = df_temp_03.columns
nomes_cols_novas = nomes_cols[1:]
print('Quantidade Total de Variáveis Criadas:', len(df_temp_03.columns) - 1)
print('Nomes das Variáveis Criadas:', nomes_cols_novas)
print('')
print('')

# Quantidade de linhas do DataFrame.
num_rows_df = df_temp_03.count()

# Quantidade de colunas do DataFrame.
num_columns_df = len(df_temp_03.columns)

# Imprimir o resultado de número de linhas e colunas.
print(f'Quantidade de linhas do DataFrame: {num_rows_df}')
print(f'Quantidade de colunas do DataFrame: {num_columns_df}')
print('')
print('')

# Mostrando o novo DataFrame com as variáveis criadas.
df_temp_03.show(5, False)

Quantidade Total de Variáveis Criadas: 720
Nomes das Variáveis Criadas: ['VL_TOT_CNT_INSTALMENT_U3M_DEMAND_POS_CASH', 'VL_MED_CNT_INSTALMENT_U3M_DEMAND_POS_CASH', 'VL_MAX_CNT_INSTALMENT_U3M_DEMAND_POS_CASH', 'VL_MIN_CNT_INSTALMENT_U3M_DEMAND_POS_CASH', 'VL_TOT_CNT_INSTALMENT_U6M_DEMAND_POS_CASH', 'VL_MED_CNT_INSTALMENT_U6M_DEMAND_POS_CASH', 'VL_MAX_CNT_INSTALMENT_U6M_DEMAND_POS_CASH', 'VL_MIN_CNT_INSTALMENT_U6M_DEMAND_POS_CASH', 'VL_TOT_CNT_INSTALMENT_U12M_DEMAND_POS_CASH', 'VL_MED_CNT_INSTALMENT_U12M_DEMAND_POS_CASH', 'VL_MAX_CNT_INSTALMENT_U12M_DEMAND_POS_CASH', 'VL_MIN_CNT_INSTALMENT_U12M_DEMAND_POS_CASH', 'VL_TOT_CNT_INSTALMENT_U24M_DEMAND_POS_CASH', 'VL_MED_CNT_INSTALMENT_U24M_DEMAND_POS_CASH', 'VL_MAX_CNT_INSTALMENT_U24M_DEMAND_POS_CASH', 'VL_MIN_CNT_INSTALMENT_U24M_DEMAND_POS_CASH', 'VL_TOT_CNT_INSTALMENT_U36M_DEMAND_POS_CASH', 'VL_MED_CNT_INSTALMENT_U36M_DEMAND_POS_CASH', 'VL_MAX_CNT_INSTALMENT_U36M_DEMAND_POS_CASH', 'VL_MIN_CNT_INSTALMENT_U36M_DEMAND_POS_CASH', 'VL_TOT_CNT_INS

### Realizando o join das tabelas criadas

In [12]:
# Fazendo o join das duas tabelas criadas.
df_temp_04 = df_temp_02.join(df_temp_03, 'SK_ID_PREV')

# Adicionando as colunas de data ao DataFrame.
df_temp_04 = df_temp_04.withColumn('PK_DATREF', date_format(current_date(), 'yyyyMMdd')) \
                       .withColumn('PK_DAT_PROC', current_date())


# Quantidade e nome das variáveis criadas.
nomes_cols = df_temp_04.columns
nomes_cols_novas = nomes_cols[1:-2]
print('Quantidade Total de Variáveis Criadas:', len(df_temp_04.columns) - 3)
print('Nomes das Variáveis Criadas:', nomes_cols_novas)
print('')
print('')

# Quantidade de linhas do DataFrame.
num_rows_df = df_temp_04.count()

# Quantidade de colunas do DataFrame.
num_columns_df = len(df_temp_04.columns)

# Imprimir o resultado de número de linhas e colunas.
print(f'Quantidade de linhas do DataFrame: {num_rows_df}')
print(f'Quantidade de colunas do DataFrame: {num_columns_df}')
print('')
print('')

# Mostrando o novo DataFrame com as variáveis criadas.
df_temp_04.show(5, False)

Quantidade Total de Variáveis Criadas: 800
Nomes das Variáveis Criadas: ['VL_TOT_CNT_INSTALMENT_U3M_POS_CASH', 'VL_MED_CNT_INSTALMENT_U3M_POS_CASH', 'VL_MAX_CNT_INSTALMENT_U3M_POS_CASH', 'VL_MIN_CNT_INSTALMENT_U3M_POS_CASH', 'VL_TOT_CNT_INSTALMENT_U6M_POS_CASH', 'VL_MED_CNT_INSTALMENT_U6M_POS_CASH', 'VL_MAX_CNT_INSTALMENT_U6M_POS_CASH', 'VL_MIN_CNT_INSTALMENT_U6M_POS_CASH', 'VL_TOT_CNT_INSTALMENT_U12M_POS_CASH', 'VL_MED_CNT_INSTALMENT_U12M_POS_CASH', 'VL_MAX_CNT_INSTALMENT_U12M_POS_CASH', 'VL_MIN_CNT_INSTALMENT_U12M_POS_CASH', 'VL_TOT_CNT_INSTALMENT_U24M_POS_CASH', 'VL_MED_CNT_INSTALMENT_U24M_POS_CASH', 'VL_MAX_CNT_INSTALMENT_U24M_POS_CASH', 'VL_MIN_CNT_INSTALMENT_U24M_POS_CASH', 'VL_TOT_CNT_INSTALMENT_U36M_POS_CASH', 'VL_MED_CNT_INSTALMENT_U36M_POS_CASH', 'VL_MAX_CNT_INSTALMENT_U36M_POS_CASH', 'VL_MIN_CNT_INSTALMENT_U36M_POS_CASH', 'VL_TOT_CNT_INSTALMENT_FUTURE_U3M_POS_CASH', 'VL_MED_CNT_INSTALMENT_FUTURE_U3M_POS_CASH', 'VL_MAX_CNT_INSTALMENT_FUTURE_U3M_POS_CASH', 'VL_MIN_CNT_INSTALME

### Salvando a tabela de POS_CASH_Balance sumarizada

In [13]:
df_temp_04.write.partitionBy("PK_DATREF").parquet('../database/feature_store/book_pos_cash')